In [2]:
# Virtual env libraries
import sys, os
sys.path.append(os.path.abspath(os.path.join('../../../.local/lib/python3.9/site-packages')))
sys.path.append(os.path.abspath(os.path.join('../../../virtualenv/lib/python3.9/site-packages')))
sys.path.append('/usr/local/sas/grid/python3-3.9.1/lib/python3.9/site-packages')

import pandas as pd
import numpy as np
from tqdm import tqdm
import saspy
import matplotlib.pyplot as plt
from multiprocessing import Pool
import seaborn as sns
import wrds
import glob
from stdnum import cusip
from pandas.tseries.offsets import MonthEnd, MonthBegin

Outline
* Set up SAS and SQL Sessions
* Prepare list of stocks for data collection
* Prepare list of dates for data collection
* For each date, grab data for the list of stocks
* Save daily hfprices to file

# Set up WRDS sessions

In [4]:
# sas = saspy.SASsession(**{'cfgname': 'default', 'encoding': 'utf_8'})
sql = wrds.Connection()

Loading library list...
Done


# Misc Data

## Get Stock Info

In [5]:
stock_info_df = pd.read_feather('../../../HFZoo/data/keys/stock_universe.feather').set_index('dt')

## Get CRSP Daily List

In [6]:
crsp_folder = '../../../HFZoo/data/crsp/daily/'
crsp_filenames = glob.glob(crsp_folder + '*.parquet')
crsp_dates = [x.split('/')[-1].split('.')[0] for x in crsp_filenames]

## Get Dates List

In [4]:
## Dates for each library
taqm_dates = list(pd.read_csv('../../data/taq/taqm_dates_list.csv').iloc[:,0])
taqmsec_dates = list(pd.read_csv('../../data/taq/taqmsec_dates_list.csv').iloc[:,0])

## Merge and filter by yyyymm input
taq_all_dates = taqm_dates + taqmsec_dates

# Functions

In [8]:
# Help convert python args to SAS code
list_to_str = lambda lst: ', '.join(["'" + x + "'" for x in lst])

# Modify SAS scripts with given args
def gen_sas_script(base_script, date_ymd, start_time, end_time, sample_interval_s, output_filename, permno_selects, cusip_selects, symbol_selects):

    base_script[10] = f"%let start_time_m = '{start_time}'t;\n"
    base_script[11] = f"%let end_time_m = '{end_time}'t;\n"
    base_script[12] = f"%let interval_seconds = {sample_interval_s};\n"
    base_script[13] = f"%let date_ymd_arg = {date_ymd};\n"
    base_script[14] = f"%let output_file  = '{output_filename}';\n"
    base_script[15] = f"%let permno_list  = ({', '.join([str(x) for x in permno_selects])});\n"
    base_script[16] = f"%let cusip_list  = ({list_to_str(cusip_selects)});\n"
    base_script[17] = f"%let symbol_list  = ({list_to_str(symbol_selects)});\n"

    # For TAQ Monthly case
    base_script[18] = f"%let date_ym_arg  = {date_ymd[:6]};\n"

    return base_script

# Main

## Params

In [9]:
# Folder params
output_folder = '/scratch/duke/sa400/SemiAddress/data_taq_prices/'
scripts_folder = '/scratch/duke/sa400/SemiAddress/scripts/'

# TAQ selection params
start_time, end_time = '9:30:00', '16:05:00'
sample_interval_s = 5*60

In [25]:
# Clean up scripts folder
files = glob.glob(f'{scripts_folder}*')
with Pool(24) as p:
    for _ in tqdm(p.imap_unordered(os.remove, files), total = len(files)):
        continue

0it [00:00, ?it/s]


## Create Scripts for Batch Job

In [26]:
def get_crsp_stock_selects(crsp_df, symbol_selects):
    # Grab permnos and cusips for stocks filtered by marketcap rank for the day
    
    # Prep CRSP dataframe
    crsp_df.columns = [x.lower() for x in crsp_df.columns]
    crsp_df['mcap']  = crsp_df['prc']*crsp_df['shrout']
    crsp_df['cusip9']  = crsp_df['cusip'].astype(str) + crsp_df['cusip'].astype(str).apply(cusip.calc_check_digit)
    crsp_df['permno'] = crsp_df['permno'].astype(int)

    # Get stocks
    permno_selects = stock_info_df.query('ticker in @symbol_selects').groupby(['permno']).first().index.values.astype(int)
    cusip_selects = crsp_df.query('permno in @permno_selects')['cusip9'].astype(str).values
    permno_selects = list(permno_selects.astype(int).astype(str))

    return permno_selects, cusip_selects

In [27]:
## Create batch scripts for running SAS programs

# Sample selection params
symbol_selects = ['SPY', 'BA', 'OXY', 'RTX', 'RTN', 'SLB', 'SPG', 'BIIB', 'BKNG', 'CHTR', 'GILD', 'FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG', 'UTX'] \
    + ['SPYV', 'VFVA', 'VFMO', 'QUAL', 'MTUM', 'SIZE', 'IWM']
permno_selects = stock_info_df.query('ticker in @symbol_selects')['permno'].unique()
symbol_selects = list(set(symbol_selects + list(stock_info_df.query('permno in @permno_selects')['ticker'].unique())))

arg_dates = list(range(2002, 2021))

for arg_date in arg_dates:
    
    print(f'Processing {arg_date}')
    arg_date = str(arg_date)
    
    # Get list of TAQ and CRSP files for given date
    k = len(arg_date)
    taq_filtered_dates = [str(x) for x in taq_all_dates if str(x)[:k] == arg_date]
    crsp_filtered_dates = [str(x) for x in crsp_dates if str(x)[:k] == arg_date]
    shared_dates = [x for x in taq_filtered_dates if x in crsp_filtered_dates]
    
    # Get base SAS script
    if int(str(arg_date)[:4]) <= 2014:
        taq_library = 'taq'
    else:
        taq_library = 'taqmsec'

    with open(f'{taq_library}_scrape.sas', 'r') as file:
        base_script = file.readlines()

    # Prepare batch script text file
    sh_file = open(f'{scripts_folder}batch_script_{arg_date}.sh', 'w+')

    # Add header to batch script file
    sh_file.write(f'''
    #!/bin/bash
    #$ -N taq_{arg_date}
    #$ -cwd\n\n''')
    
    # Generate SAS scripts for each day
    def update_sh_file(date_str):
        
        # Skip if weekend
        if pd.to_datetime(date_str).weekday() > 4:
            return ''
        
        # Get CRSP file 
        try: 
            date_crsp_file = pd.read_parquet(crsp_folder + date_str + '.parquet',
                                           columns = ['prc', 'cusip', 'date', 'permno', 'permco', 'shrout'])
        except:
            print(date_str)
            raise Exception
        
        # Determine appropriate stocks
        permno_selects, cusip_selects = get_crsp_stock_selects(date_crsp_file, symbol_selects)
        
        if len(permno_selects)*len(cusip_selects) == 0:
            print('Missing PERMNOs and CUSIPs for', date_str)
            return ''
        
        # Set up output filename
        output_filename = output_folder + f'{date_str}.csv'

        # Generate SAS script for data collection
        file_lines = gen_sas_script(base_script, date_str, start_time, end_time, sample_interval_s, 
                                    output_filename, permno_selects, cusip_selects, symbol_selects)

        # Write SAS script to file
        with open(scripts_folder + f'{date_str}.sas', 'w+') as sas_script_file:
            sas_script_file.writelines(file_lines)

        # Create SAS script reference to .sh file
        sas_reference = f'sas {date_str}.sas \n'

        return sas_reference

    with Pool(24) as p:
        for sh_command in tqdm(p.imap_unordered(update_sh_file, shared_dates), total = len(shared_dates)):
            sh_file.write(sh_command)

    sh_file.close()

Processing 2002


100%|██████████| 252/252 [00:23<00:00, 10.91it/s]


Processing 2003


100%|██████████| 252/252 [00:22<00:00, 11.02it/s]


Processing 2004


...


100%|██████████| 251/251 [00:23<00:00, 10.72it/s]


Processing 2019


100%|██████████| 252/252 [00:24<00:00, 10.50it/s]


Processing 2020


100%|██████████| 253/253 [00:23<00:00, 10.74it/s]


In [29]:
## Get list of .sh file commands
sh_files = list(np.sort([y for y in [x for x in os.walk(scripts_folder)][0][2] if '.sh' in y and '.sh.' not in y]))[::-1]
for f in sh_files:
    print('qsub -cwd', f)
print('\n')

qsub -cwd batch_script_2020.sh
qsub -cwd batch_script_2019.sh
qsub -cwd batch_script_2018.sh
qsub -cwd batch_script_2017.sh
qsub -cwd batch_script_2016.sh
qsub -cwd batch_script_2015.sh
qsub -cwd batch_script_2014.sh
qsub -cwd batch_script_2013.sh
qsub -cwd batch_script_2012.sh
qsub -cwd batch_script_2011.sh
qsub -cwd batch_script_2010.sh
qsub -cwd batch_script_2009.sh
qsub -cwd batch_script_2008.sh
qsub -cwd batch_script_2007.sh
qsub -cwd batch_script_2006.sh
qsub -cwd batch_script_2005.sh
qsub -cwd batch_script_2004.sh
qsub -cwd batch_script_2003.sh
qsub -cwd batch_script_2002.sh




# Reformat output

In [36]:
new_output_folder = '../../data/taq/prices/'

In [37]:
# Clean up new output folder
files = glob.glob(f'{new_output_folder}*')
for f in tqdm(files):
    os.remove(f)

100%|██████████| 4784/4784 [00:02<00:00, 2317.16it/s]


In [38]:
def reformat_taq_files(date):
    
    # Get all files for date
    files_date = glob.glob(output_folder + date + '*.csv')
    taq_df = pd.concat([pd.read_csv(file) for file in files_date], ignore_index = True).reset_index(drop = True)
    
    # Clean up and save
    taq_df.columns = [x.lower() for x in taq_df.columns]
    taq_df[['permno', 'cusip9', 'symbol']] = taq_df['ticker_identifier'].str.split('_', expand = True)
    taq_df['permno'] = taq_df['permno'].str.replace('.', '', regex = False)
    taq_df = taq_df.astype({x:'category' for x in ['permno', 'cusip9', 'symbol', 'ticker_identifier']})
    taq_df = taq_df.sort_index(axis=1)
    taq_df.to_parquet(f'{new_output_folder}{date}.parquet')

# Get list of existing files
taq_files = glob.glob(output_folder + '*.csv')
taq_dates = [x.split('/')[-1].split('.')[0].split('_')[0] for x in taq_files]
taq_dates = list(set(taq_dates))

with Pool(24) as p:
    for _ in tqdm(p.imap_unordered(reformat_taq_files, taq_dates), total = len(taq_dates)):
        continue

100%|██████████| 4784/4784 [00:50<00:00, 95.50it/s] 


# Check output

In [10]:
output_filelocs = glob.glob(output_folder + '*.csv')
output_filenames = [x.split('/')[-1].split('.')[0] for x in output_filelocs]
output_filedates = [x.split('/')[-1].split('_')[0] for x in output_filelocs]

In [11]:
output_years = np.sort(list(set([x[:4] for x in output_filenames])))
output_suffixes = np.sort(list(set([x[9:] for x in output_filenames])))

print('Years: ' + ', '.join(output_years))
print('Suffixes: ' + ', '.join(output_suffixes))

for year_filter in output_years:
    
    output_filenames_year = [x for x in output_filenames if x[:4] == year_filter]
    print('='*100)
    print(f'[{year_filter}]')
    print('='*100)
    

    for suffix_filter in output_suffixes:
        print(f'[{suffix_filter}]')
        output_filenames_year_suffix = [x for x in output_filenames_year if x[9:] == suffix_filter]
        output_filenames_year_suffix_dates = [x[:8] for x in output_filenames_year_suffix]
        print('\tFiles: ', len(output_filenames_year_suffix))
        print('\tMissing files: ', end = '\n\t\t')

        for date in np.sort([str(x) for x in taq_all_dates if str(x)[:4] == str(year_filter)]):
            if date not in output_filenames_year_suffix_dates:
                weekend_label = " (weekend)" if pd.to_datetime(date).weekday() > 4 else ""
                print(f'{date}{weekend_label}', end = ', ')
        print('\n')

Years: 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
Suffixes: 
[2002]
[]
	Files:  252
	Missing files: 
		

[2003]
[]
	Files:  252
	Missing files: 
		

[2004]
[]
	Files:  252
	Missing files: 
		

[2005]
[]
	Files:  252
	Missing files: 
		

[2006]
[]
	Files:  251
	Missing files: 
		

[2007]
[]
	Files:  251
	Missing files: 
		

[2008]
[]
	Files:  253
	Missing files: 
		

[2009]
[]
	Files:  252
	Missing files: 
		

[2010]
[]
	Files:  252
	Missing files: 
		

[2011]
[]
	Files:  252
	Missing files: 
		

[2012]
[]
	Files:  250
	Missing files: 
		

[2013]
[]
	Files:  252
	Missing files: 
		

[2014]
[]
	Files:  252
	Missing files: 
		

[2015]
[]
	Files:  252
	Missing files: 
		

[2016]
[]
	Files:  252
	Missing files: 
		

[2017]
[]
	Files:  251
	Missing files: 
		

[2018]
[]
	Files:  251
	Missing files: 
		20181230 (weekend), 

[2019]
[]
	Files:  252
	Missing files: 
		20190106 (weekend), 20190810 (weekend), 

[2020]
[]
	Files: